In [3]:
from deepmash.data_processing.gtzan_stems import GTZANStemsDataset 

# reload imported modules automatically (so you dont have to restart kernel when changing .py files)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
GTZANStemsDataset(
    root_dir="datasets/gtzan-stems",
    preprocess=True
)

Preprocessing GTZAN stems from datasets/gtzan-stems to datasets/gtzan-stems-processed ...


  0%|          | 0/1000 [00:00<?, ?it/s]